In [1]:
!pip install tensorflow_datasets

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [3]:
AUTOTUNE = tf.data.AUTOTUNE

#wczytanie danych
(train_ds,test_ds),info = tfds.load(
    "cifar10",
    split=['train','test'],
    as_supervised=True,
    with_info=True
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/incomplete.ICX6NO_3.0.2/cifar10-train.tfrecord*...:   0%|         …

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar10/incomplete.ICX6NO_3.0.2/cifar10-test.tfrecord*...:   0%|          …

Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.2. Subsequent calls will reuse this data.


In [4]:
for image,labels in train_ds.take(1):
    print(f"Images shape: {image.shape}")
    print(f"Labels shape: {labels.shape}")

Images shape: (32, 32, 3)
Labels shape: ()


In [5]:
#normalizacja danych
def preprocess(image,label):
    image = tf.cast(image,tf.float32)/255.0
    return image,label

In [6]:
BATCH_SIZE=128

In [7]:
train_ds = (
    train_ds
    .shuffle(10000)
    .map(preprocess,num_parallel_calls=AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE)
)

test_ds = (
    test_ds
    .map(preprocess,num_parallel_calls=AUTOTUNE)
    .batch(BATCH_SIZE)
    .prefetch(AUTOTUNE)
)

In [8]:
for image,labels in train_ds.take(1):
    print(f"Images shape: {image.shape}")
    print(f"Labels shape: {labels.shape}")

Images shape: (128, 32, 32, 3)
Labels shape: (128,)


In [9]:
#prosta sieć neuronowa
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(32,32,3)),
    tf.keras.layers.Dense(256,activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(10,activation='softmax')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [10]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

In [11]:
#trenujemy model
model.fit(train_ds,epochs=10,validation_data=test_ds)

Epoch 1/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 19s 41ms/step - accuracy: 0.2618 - loss: 2.0561 - val_accuracy: 0.3868 - val_loss: 1.7312
Epoch 2/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 11s 28ms/step - accuracy: 0.3809 - loss: 1.7323 - val_accuracy: 0.4193 - val_loss: 1.6348
Epoch 3/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 11s 28ms/step - accuracy: 0.4202 - loss: 1.6323 - val_accuracy: 0.4362 - val_loss: 1.5705
Epoch 4/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 12s 30ms/step - accuracy: 0.4434 - loss: 1.5635 - val_accuracy: 0.4376 - val_loss: 1.5892
Epoch 5/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 20s 29ms/step - accuracy: 0.4626 - loss: 1.5258 - val_accuracy: 0.4631 - val_loss: 1.5060
Epoch 6/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 20s 27ms/step - accuracy: 0.4708 - loss: 1.4859 - val_accuracy: 0.4690 - val_loss: 1.5057
Epoch 7/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 11s 27ms/step - accuracy: 0.4815 - loss: 1.4632 - val_accuracy: 0.4848 - val_loss: 1.4597
Epoch 8/10
391/391 ━━━━━━━━━━━━━━━━━━━━ 21s 28ms/step - accuracy: 0.4921 - loss: 1.4295 - 

In [12]:
#ocena modelu - ewaluacja
test_loss,test_acc = model.evaluate(test_ds)
print(f"Test accuracy: {test_acc}")

79/79 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.4993 - loss: 1.4109
Test accuracy: 0.5
